In [1]:
from vnpy.trader.constant import Exchange,Interval
from data_analysis import DataAnalysis
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
datetime(2021, 6, 15)

datetime.datetime(2021, 6, 15, 0, 0)

In [3]:
datetime.now()

datetime.datetime(2021, 6, 14, 16, 54, 33, 615800)

In [4]:
herramiento = DataAnalysis()
herramiento.load_history(    
    symbol="ethusdt", 
    exchange=Exchange.BINANCE, 
    interval=Interval.MINUTE, 
    start=datetime(2021, 5, 3), 
    end=datetime.now(),
    rate = 8/10000,
    index_3to1 = ["ATR","CCI"],
    index_1to1 = ["STDDEV","SMA"],
    index_2to2 = ["AROON"],
    index_2to1 = ["AROONOSC"],
    index_4to1 = ["BOP"],
    window_index=30,
)

2021-06-14 16:54:33.632756	开始加载历史数据
2021-06-14 16:54:38.389895	历史数据加载完成，数据量：61495


In [5]:
herramiento.orignal

,open,high,low,close,volume
2021-05-03 00:00:00+08:00,2929.14,2937.27,2929.14,2936.33,782.78806
2021-05-03 00:01:00+08:00,2936.33,2937.62,2933.79,2934.00,799.98135
2021-05-03 00:02:00+08:00,2934.28,2934.50,2927.00,2928.33,308.29920
2021-05-03 00:03:00+08:00,2928.32,2928.94,2925.00,2925.18,404.87646
2021-05-03 00:04:00+08:00,2925.26,2926.84,2924.13,2924.83,320.95382
...,...,...,...,...,...
2021-06-14 16:50:00+08:00,2473.23,2477.47,2473.22,2476.81,545.61010
2021-06-14 16:51:00+08:00,2476.70,2478.09,2475.72,2476.87,372.65066
2021-06-14 16:52:00+08:00,2476.92,2476.96,2472.75,2476.13,339.54627
2021-06-14 16:53:00+08:00,2476.13,2477.77,2473.70,2477.21,351.33906


In [6]:
df = herramiento.orignal
df.head()

,open,high,low,close,volume
2021-05-03 00:00:00+08:00,2929.14,2937.27,2929.14,2936.33,782.78806
2021-05-03 00:01:00+08:00,2936.33,2937.62,2933.79,2934.00,799.98135
2021-05-03 00:02:00+08:00,2934.28,2934.50,2927.00,2928.33,308.29920
2021-05-03 00:03:00+08:00,2928.32,2928.94,2925.00,2925.18,404.87646
2021-05-03 00:04:00+08:00,2925.26,2926.84,2924.13,2924.83,320.95382


In [7]:
df.tail()

,open,high,low,close,volume
2021-06-14 16:50:00+08:00,2473.23,2477.47,2473.22,2476.81,545.61010
2021-06-14 16:51:00+08:00,2476.70,2478.09,2475.72,2476.87,372.65066
2021-06-14 16:52:00+08:00,2476.92,2476.96,2472.75,2476.13,339.54627
2021-06-14 16:53:00+08:00,2476.13,2477.77,2473.70,2477.21,351.33906
2021-06-14 16:54:00+08:00,2477.21,2477.22,2475.43,2476.08,51.62346


#### 合成半小时K线数据
data30 = pd.DataFrame()
interval='30T'
data30["open"] = df["open"].resample(interval).first()
data30["high"] = df["high"].resample(interval).max()
data30["low"] = df["low"].resample(interval).min()
data30["close"] = df["close"].resample(interval).last()
data30["volume"] = df["volume"].resample(interval).sum()

参数配置

In [8]:
#标准差
STD = 0.25

#滑动窗口小时数
rolling_hours_window = 2

# 套利摩擦
arb_fri = 0.006

# 千三池 0.006， 万五池 0.001
bar_len = 0.001



In [9]:
len_data = len(df["close"])

In [10]:
len_data

61495

In [11]:
df.iloc[0]

open      2929.14000
high      2937.27000
low       2929.14000
close     2936.33000
volume     782.78806
Name: 2021-05-03 00:00:00+08:00, dtype: float64

In [12]:
df["pre_close"] = df["close"].shift(1).fillna(0)

In [13]:
df['log_return'] = np.log(df['close']/df['pre_close'])

In [14]:
df['log_return'].rolling(30).std()

2021-05-03 00:00:00+08:00         NaN
2021-05-03 00:01:00+08:00         NaN
2021-05-03 00:02:00+08:00         NaN
2021-05-03 00:03:00+08:00         NaN
2021-05-03 00:04:00+08:00         NaN
                               ...   
2021-06-14 16:50:00+08:00    0.000844
2021-06-14 16:51:00+08:00    0.000844
2021-06-14 16:52:00+08:00    0.000836
2021-06-14 16:53:00+08:00    0.000841
2021-06-14 16:54:00+08:00    0.000841
Name: log_return, Length: 61495, dtype: float64

In [15]:
df['rolling_sigma'] = df['log_return'].rolling(rolling_hours_window*60).std()
df['daily_sigma'] = df['rolling_sigma']*np.sqrt(1440)

In [16]:
df.iloc[119:125]

,open,high,low,close,volume,pre_close,log_return,rolling_sigma,daily_sigma
2021-05-03 01:59:00+08:00,2934.03,2934.15,2932.67,2933.16,159.57674,2934.04,-0.000300,NaN,NaN
2021-05-03 02:00:00+08:00,2933.17,2934.76,2933.16,2934.16,166.72626,2933.16,0.000341,0.000657,0.024925
2021-05-03 02:01:00+08:00,2934.16,2934.17,2932.67,2933.50,131.26980,2934.16,-0.000225,0.000653,0.024785
2021-05-03 02:02:00+08:00,2933.51,2934.70,2932.02,2932.12,201.46676,2933.50,-0.000471,0.000630,0.023907
2021-05-03 02:03:00+08:00,2932.12,2932.12,2930.06,2931.54,180.87564,2932.12,-0.000198,0.000622,0.023616
2021-05-03 02:04:00+08:00,2931.54,2937.27,2931.54,2937.14,350.06458,2931.54,0.001908,0.000646,0.024501


In [17]:
df['up_float'] = np.exp(df['daily_sigma']*STD)
df['down_float'] = np.exp(-df['daily_sigma']*STD)

In [18]:
df['target_range'] = np.round(np.log(df['up_float'])/np.log(1+bar_len),0)

In [19]:
df['target_range'].max()

237.0

In [20]:
df['target_range'].min()

5.0

In [21]:
df.index = df.index.tz_localize(None)

In [22]:
df.index

DatetimeIndex(['2021-05-03 00:00:00', '2021-05-03 00:01:00',
               '2021-05-03 00:02:00', '2021-05-03 00:03:00',
               '2021-05-03 00:04:00', '2021-05-03 00:05:00',
               '2021-05-03 00:06:00', '2021-05-03 00:07:00',
               '2021-05-03 00:08:00', '2021-05-03 00:09:00',
               ...
               '2021-06-14 16:45:00', '2021-06-14 16:46:00',
               '2021-06-14 16:47:00', '2021-06-14 16:48:00',
               '2021-06-14 16:49:00', '2021-06-14 16:50:00',
               '2021-06-14 16:51:00', '2021-06-14 16:52:00',
               '2021-06-14 16:53:00', '2021-06-14 16:54:00'],
              dtype='datetime64[ns]', length=61495, freq=None)

In [23]:
df.index[10:12]

DatetimeIndex(['2021-05-03 00:10:00', '2021-05-03 00:11:00'], dtype='datetime64[ns]', freq=None)

In [24]:
#df.to_excel('test.xlsx')

In [25]:
df['close_rolling_30_max'] = df['close'].rolling(30).max()
df['close_rolling_30_min'] = df['close'].rolling(30).min()

In [26]:
# 初始化
start_index = '2021-05-03 02:00:00'
df.loc[start_index,'adjust_signal'] = 1
df.loc[start_index,'actural_range'] = df.loc[start_index,'target_range']
df.loc[start_index,'benchmark_price'] = df.loc[start_index]['pre_close']
df.loc[start_index,'a'] = df.loc[start_index]['benchmark_price']*df.loc[start_index]['down_float']*(1-arb_fri)
df.loc[start_index,'b'] = df.loc[start_index]['benchmark_price']*df.loc[start_index]['up_float']*(1+arb_fri)
a  = df.loc[start_index]['benchmark_price']*df.loc[start_index]['down_float']*(1-arb_fri)
b = df.loc[start_index]['benchmark_price']*df.loc[start_index]['up_float']*(1+arb_fri)

df_1 = df[start_index:]
#df_1['close_rolling_30_max'] = df_1['close'].rolling(30).max()
#df_1['close_rolling_30_min'] = df_1['close'].rolling(30).min()
data_len = len(df_1)

start_index_list=[]

In [27]:
df_1

,open,high,low,close,volume,pre_close,log_return,rolling_sigma,daily_sigma,up_float,down_float,target_range,close_rolling_30_max,close_rolling_30_min,adjust_signal,actural_range,benchmark_price,a,b
2021-05-03 02:00:00,2933.17,2934.76,2933.16,2934.16,166.72626,2933.16,0.000341,0.000657,0.024925,1.006251,0.993788,6.0,2934.73,2928.06,1.0,6.0,2933.16,2897.449637,2969.203589
2021-05-03 02:01:00,2934.16,2934.17,2932.67,2933.50,131.26980,2934.16,-0.000225,0.000653,0.024785,1.006216,0.993823,6.0,2934.73,2928.06,NaN,NaN,NaN,NaN,NaN
2021-05-03 02:02:00,2933.51,2934.70,2932.02,2932.12,201.46676,2933.50,-0.000471,0.000630,0.023907,1.005995,0.994041,6.0,2934.73,2928.30,NaN,NaN,NaN,NaN,NaN
2021-05-03 02:03:00,2932.12,2932.12,2930.06,2931.54,180.87564,2932.12,-0.000198,0.000622,0.023616,1.005921,0.994113,6.0,2934.73,2928.62,NaN,NaN,NaN,NaN,NaN
2021-05-03 02:04:00,2931.54,2937.27,2931.54,2937.14,350.06458,2931.54,0.001908,0.000646,0.024501,1.006144,0.993893,6.0,2937.14,2928.62,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-14 16:50:00,2473.23,2477.47,2473.22,2476.81,545.61010,2473.11,0.001495,0.001027,0.038973,1.009791,0.990304,10.0,2487.11,2470.73,NaN,NaN,NaN,NaN,NaN
2021-06-14 16:51:00,2476.70,2478.09,2475.72,2476.87,372.65066,2476.81,0.000024,0.001027,0.038958,1.009787,0.990308,10.0,2487.11,2470.73,NaN,NaN,NaN,NaN,NaN
2021-06-14 16:52:00,2476.92,2476.96,2472.75,2476.13,339.54627,2476.87,-0.000299,0.000854,0.032392,1.008131,0.991935,8.0,2487.11,2470.73,NaN,NaN,NaN,NaN,NaN
2021-06-14 16:53:00,2476.13,2477.77,2473.70,2477.21,351.33906,2476.13,0.000436,0.000847,0.032123,1.008063,0.992001,8.0,2487.11,2470.73,NaN,NaN,NaN,NaN,NaN


In [28]:
for i in range(1,data_len):
    if (df_1['close_rolling_30_max'][i]<a) | (df_1['close_rolling_30_min'][i]>b):
        df_1['adjust_signal'][i] = 1
        df_1['benchmark_price'][i] = df_1['pre_close'][i]
        
        df_1['a'][i] = df_1['benchmark_price'][i]*df_1['down_float'][i]*(1-arb_fri)
        df_1['b'][i] = df_1['benchmark_price'][i]*df_1['up_float'][i]*(1+arb_fri)
        a = df_1['benchmark_price'][i]*df_1['down_float'][i]*(1-arb_fri)
        b = df_1['benchmark_price'][i]*df_1['up_float'][i]*(1+arb_fri)
        
        df_1['actural_range'][i] = np.round(np.log(df_1['up_float'][i])/np.log(1+bar_len),0)
        start_index = df_1.index[i]
        start_index_list.append(start_index)
        #break
    else:
        df_1['adjust_signal'][i] = 0
        df_1['benchmark_price'][i] = df_1['benchmark_price'][start_index]
        df_1['a'][i] =  df_1['a'][start_index]
        df_1['b'][i] =  df_1['b'][start_index]
        df_1['actural_range'][i] =  df_1['target_range'][start_index]
        continue

In [29]:
start_index_list

[Timestamp('2021-05-03 05:50:00'),
 Timestamp('2021-05-03 10:44:00'),
 Timestamp('2021-05-03 13:45:00'),
 Timestamp('2021-05-03 16:41:00'),
 Timestamp('2021-05-03 20:52:00'),
 Timestamp('2021-05-04 01:03:00'),
 Timestamp('2021-05-04 06:23:00'),
 Timestamp('2021-05-04 10:22:00'),
 Timestamp('2021-05-04 19:00:00'),
 Timestamp('2021-05-04 23:34:00'),
 Timestamp('2021-05-06 04:04:00'),
 Timestamp('2021-05-06 23:42:00'),
 Timestamp('2021-05-07 03:05:00'),
 Timestamp('2021-05-08 21:39:00'),
 Timestamp('2021-05-09 00:35:00'),
 Timestamp('2021-05-09 05:24:00'),
 Timestamp('2021-05-09 14:55:00'),
 Timestamp('2021-05-09 18:37:00'),
 Timestamp('2021-05-09 20:49:00'),
 Timestamp('2021-05-10 00:27:00'),
 Timestamp('2021-05-10 11:41:00'),
 Timestamp('2021-05-10 16:59:00'),
 Timestamp('2021-05-10 19:15:00'),
 Timestamp('2021-05-11 00:06:00'),
 Timestamp('2021-05-11 04:17:00'),
 Timestamp('2021-05-11 18:32:00'),
 Timestamp('2021-05-11 20:23:00'),
 Timestamp('2021-05-12 02:39:00'),
 Timestamp('2021-05-

In [30]:
df_1['adjust_signal'].sum()

128.0

In [31]:
#df_1.to_excel('test3.xlsx')